In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import xarray as xr
import xclim as xc
import pyhomogenize as pyh
import warnings

In [3]:
ifile='/pool/data/CORDEX/data/cordex/output/EUR-11/GERICS/MPI-M-MPI-ESM-LR/historical/r3i1p1/GERICS-REMO2015/v1/day/tas/v20190925/tas_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_20010101-20051231.nc'

In [4]:
def open_xrdataset(files, use_cftime=True, parallel=True, data_vars='minimal', chunks={'time':1}, 
                   coords='minimal', compat='override', drop=None, **kwargs):
    """optimized function for opening large cf datasets.
    based on https://github.com/pydata/xarray/issues/1385#issuecomment-561920115
    decode_timedelta=False is added to leave variables and coordinates with time units in 
    {“days”, “hours”, “minutes”, “seconds”, “milliseconds”, “microseconds”} encoded as numbers.   
    """
    def drop_all_coords(ds):
        return ds.reset_coords(drop=True)

    ds = xr.open_mfdataset(files, parallel=parallel, decode_times=False, combine='by_coords', 
                           preprocess=drop_all_coords, decode_cf=False, chunks=chunks,
                           data_vars=data_vars, coords=coords, compat=compat, **kwargs)

    return xr.decode_cf(ds, use_cftime=use_cftime, decode_timedelta=False)

In [5]:
ds = open_xrdataset(ifile)
ds

<xarray.Dataset>
Dimensions:                     (time: 1826, bnds: 2, rlat: 412, rlon: 424,
                                 vertices: 4)
Coordinates:
  * time                        (time) object 2001-01-01 12:00:00 ... 2005-12...
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    height                      float64 ...
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds                   (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    tas                         (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
Attributes: (12/34)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    title:                          GERICS-REMO2015 model output prepared for...
    parent_experiment:              N/A
    modeling_realm:                 atmos
    realization:                    3
    cmor_version:                   2.9.1
    tracking_id:                    hdl:21.14103/55a844fd-666d-4799-8563-8fbd...

In [6]:
_freq = {'year': 'YS',
         'sem' : 'QS-DEC', 
         'mon' : 'MS',
         'day' : 'D'}

_tfreq = {'year': ['YS','Y'],
          'sem' : ['QS-DEC','Q-FEB'],
          'mon' : ['MS','M'],
          'day' : 'D'}

_bounds = {'day'   : {'start' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],                       
                      'end'   : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]},            
           'year'  : {'start' : [1],                       
                      'end'   : [12]},            
           'sem'   : {'start' : [3, 6, 9, 12],                       
                      'end'   : [2, 5, 8, 11]},            
           'month' : {'start' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],                       
                      'end'   : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]},            
           'fx'    : {'start' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],                       
                      'end'   : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}} 

_fmt = {'day'   : '%Y%m%d',         
        'year'  : '%Y' ,         
        'month' : '%Y%m' ,         
        'sem'   : '%Y%m' ,         
        'fx'    : '%Y%m%d' }

In [7]:
import json
with open('../tables/indices.json', 'r') as f:
  ijson = json.load(f)
with open('../tables/xcalc.json', 'r') as f:
  xjson = json.load(f)
with open('../tables/projects.json', 'r') as f:
  pjson = json.load(f)

In [25]:
import numpy as np

class indices:
    def __init__(self, ds, 
                 index,
                 project,
                 institution,
                 institution_id,
                 var_name=None,
                 freq='year',
                 period=None,
                 base_period_time_range=None,
                 time_range=None, 
                 check_time_axis=True, 
                 crop_time_axis=True, 
                 write=False,
                 ):
        
        self.ds   = ds
        self.CIname = index
        self.project = project
        self.institution = institution
        self.institution_id = institution_id
        self.var_name = var_name
        self.freq = freq
        self.fmt = _fmt[freq]
        self.afmt = _fmt[ds.frequency]
        self.period = period
        self.base_period_time_range = base_period_time_range
        self.time_range = time_range
        self.check_time_axis = check_time_axis
        self.crop_time_axis = crop_time_axis
        self.write = write
        
    def _outname(self):
        drs = {}
        try:
            drs['output_fmt']   = pjson['climdex'+self.project]['format']
            drs['output_comps'] = pjson['climdex'+self.project]['components'].split(', ')
        except:
            warnings.warn('Project climdex{} no known'.format(self.project))
            return
        ocomps = []
        for comp in drs['output_comps']:
            if hasattr(self.idx_ds, comp):
                ocomps.append(getattr(self.idx_ds, comp))         
            elif hasattr(self.ds, comp): 
                ocomps.append(getattr(self.ds, comp))
            else:
                ocomps.append('NA')
                warnings.warn('{} not found!'.format(comp))
        return drs['output_fmt'].format(*ocomps)
        
    def _get_time_range_as_str(self, time, fmt):
        basics = pyh.basics()
        ts = basics.date_to_str(time[0], fmt)
        te = basics.date_to_str(time[-1], fmt)
        return [ts, te]
    
    def preprocessing(self):
        time_control = pyh.time_control(self.ds)
        if not self.var_name:
            self.var_name=time_control.get_var_name()
        avail_time = self._get_time_range_as_str(time_control.time, self.afmt)
        
        if self.time_range:
            time_control.select_time_range(self.time_range)
        if self.crop_time_axis:
            time_control.select_limited_time_range(smonth=_bounds[self.freq]['start'], 
                                                   emonth=_bounds[self.freq]['end'])   
        req_time = self._get_time_range_as_str(time_control.time, self.fmt)
        
        if self.check_time_axis:
            time_control.check_timestamps(correct=True)
        self.TimeRange  = req_time
        self.ATimeRange = avail_time
        
        return time_control.ds
    
    def processing(self):
        array = getattr(self, self.CIname)()
        basics = pyh.basics()
        date_range = basics.date_range(start=self.preproc.time.values[0],
                                       end=self.preproc.time.values[-1],
                                       frequency=_tfreq[self.freq])
        array = array.assign_coords({'time' : date_range})
        data_vars = {k: self.preproc.data_vars[k] for k in self.preproc.data_vars.keys() if k not in self.var_name}
        data_vars[self.CIname] = array
        data_vars['time'] = array['time']
        del data_vars['time_bnds']
        idx_ds = xr.Dataset(data_vars=data_vars, 
                            attrs=self.preproc.attrs)
        idx_ds = idx_ds.assign_coords({'time' : np.asarray(idx_ds.time, 'datetime64[ns]')})
        idx_ds = idx_ds.cf.add_bounds('time')
        idx_ds = idx_ds.reset_coords('time_bounds')
        idx_ds['time_bounds'] = idx_ds.time_bounds.transpose()
        return idx_ds.rename({'time_bounds':'time_bnds'})

    def postprocessing(self):

        def adjust_attributes(dictionary, value):
            output = {}
            for key in dictionary.keys():
                if isinstance(dictionary[key], dict):
                    output[key] = adjust_attributes(dictionary[key], value)
                else:
                    output[key] = dictionary[key].format(value)
            return output
        
        json = {}
        json[self.CIname] = ijson[self.CIname]
        json[self.CIname].update(xjson['variable_att'])
        json['global_att'] = xjson['global_att']
        try:
            json['global_att'].update(xjson[self.project]['global_att'])       
        except:
            warnings.warn('Project {} not known.'.format(self.project))
        self.json = adjust_attributes(json, None)
        
        from ci_netcdfattributes import NetCDFvariableattributes, NetCDFglobalattributes
        
        output = NetCDFvariableattributes(self, self.idx_ds[self.CIname], self.json[self.CIname]).output
        output = NetCDFglobalattributes(self, self.idx_ds, self.json['global_att']).output

        return output
    
    def to_netcdf(self):
        MISSVAL=1e20
        encoding={self.CIname:{'_FillValue':MISSVAL, 'missing_value':MISSVAL}, 
                  'time':{'units': self.ds.time.encoding['units'],
                          'calendar': self.ds.time.encoding['calendar'], 
                          'dtype': self.ds.time.encoding['dtype']},
                  'time_bnds':{'units': self.ds.time_bnds.encoding['units'],
                               'calendar': self.ds.time_bnds.encoding['calendar'], 
                               'dtype': self.ds.time.encoding['dtype']}}
        self.postproc.to_netcdf(self.outname, encoding=encoding, format='NETCDF4', unlimited_dims={'time':True})
        print('File written: {}'.format(self.outname))
        
    def compute(self, output=False):
        self.preproc = self.preprocessing()
        self.idx_ds  = self.processing()
        self.postproc = self.postprocessing()
        write = False
        if output is True:
            self.outname = self._outname()
            if self.outname: 
                write = True
            else:
                warnings.warn('Could not write output.')
        elif isinstance(output, str):
            self.outname = output
            write = True
        if write: self.to_netcdf()
            
        return self.postproc
    
    def TG(self):
        return xc.atmos.tg_mean(ds=self.preproc, freq=_freq[self.freq])

In [27]:
tg = indices(ds, index='TG', 
             freq='sem', 
             project='CORDEX', 
             institution='Helmholtz-Zentrum hereon GmbH, Climate Service Center Germany', 
             institution_id='GERICS').compute(output=True)
tg

/home/g/g300100/.conda/envs/index_calculator/lib/python3.10/site-packages/xarray/coding/times.py:351: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  sample = dates.ravel()[0]


File written: TG_EUR-11_MPI-M-MPI-ESM-LR_historical_r3i1p1_GERICS-REMO2015_v1_day_GERICS_sem_200103-200511.nc


<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4, time: 19,
                                 bounds: 2)
Coordinates:
    height                      float64 2.0
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 21.99 22.03 ... 66.75 66.69
    lon                         (rlat, rlon) float32 349.9 350.0 ... 64.76 64.96
  * time                        (time) datetime64[ns] 2001-04-16T12:00:00 ......
Dimensions without coordinates: vertices, bounds
Data variables:
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    TG                          (time, rlat, rlon) float32 dask.array<chunksize=(1, 412, 424), meta=np.ndarray>
    time_bnds                   (time, bounds) datetime64[ns] 2001-03-01T12:0...
Attributes: (12/45)
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    institute_id:                   GERICS
    experiment_id:                  historical
    source:                         GERICS-REMO2015
    model_id:                       GERICS-REMO2015
    forcing:                        N/A
    ...                             ...
    ci_package_name:                xclim_0.35.0
    ci_package_reference:           xcalc_v0.1.0
    ci_product:                     rcm_derived
    ci_frequency:                   sem
    ci_timerange_index:             200103-200511
    ci_timerange_source:            20010101-20051231